In [1]:
# Core analysis packages
import numpy as np
import os, sys
import pandas as pd
from scipy import stats
from scipy.special import comb
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import anova
import bff
import matplotlib.pyplot as plt
plt.rcdefaults()
import seaborn as sns 
sns.set(style="ticks", color_codes=True)
sns.set_style("white")
sns.set_style({'xtick.bottom': True, 'ytick.left': True})
colorref = ["gray", "royalblue", "crimson", "goldenrod", "mediumorchid", "seagreen"]

# iPython magic commands
%matplotlib notebook
%load_ext autoreload
%autoreload 2
%autosave 30

SMALL_SIZE = 12
MEDIUM_SIZE = 12
BIG_SIZE = 14

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIG_SIZE)  # fontsize of the figure title
cust_palette = sns.color_palette("Paired")[6:10]
cust_palette = [cust_palette[i] for i in [1,0,3,2]]

def median_split(S):
    return S > S.median()

Autosaving every 30 seconds


In [2]:
df = pd.read_excel("data60.xlsx")
df

,ResponseId,Country,cond,condName,Intervention_order,Intro_Timer,Intervention_Timer,BELIEFcc,POLICYcc,SHAREcc,...,Trust_sci1_1.1,control_timer2_First Click,control_timer2_Last Click,control_timer2_Page Submit,control_timer2_Click Count,Trust_sci2_1.1,control_timer3_First Click,control_timer3_Last Click,control_timer3_Page Submit,control_timer3_Click Count
0,R_0031kFvKFnFD4E9,Israel,7,PsychDistance,SocialmediasharingPipedText|PolicySupport|Beli...,4.801,104.031,92.75,86.888889,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,R_0059fW1EtKYsTWV,Brazil,5,CollectAction,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,11.155,198.395,100.00,100.000000,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R_00688n4RYG3zIel,Macedonia,10,DynamicNorm,SocialmediasharingPipedText|PolicySupport|Beli...,29.154,83.853,100.00,54.000000,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,R_0086kfWbVfurc1H,France,8,PluralIgnorance,PolicySupport|SocialmediasharingPipedText|Beli...,8.301,29.399,80.25,85.222222,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,R_00B8XyVmrMofqGB,UK,8,PluralIgnorance,SocialmediasharingPipedText|PolicySupport|Beli...,17.951,148.918,51.00,48.777778,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54653,R_zZMtNlzas0E8RyN,Slovakia,9,LetterFuture,BeliefinAnthrCC|PolicySupport|Socialmediashari...,31.481,316.675,98.50,91.333333,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54654,R_zZPllZfBJWjfSCZ,Switzerland,9,LetterFuture,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,5.627,206.393,63.00,47.444444,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54655,R_zZPOdDw0XWugcH7,Israel,10,DynamicNorm,PolicySupport|SocialmediasharingPipedText|Beli...,16.698,104.157,98.75,63.888889,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54656,R_zZpwx0yCZNh32ql,Algeria,10,DynamicNorm,BeliefinAnthrCC|PolicySupport|Socialmediashari...,17.577,79.966,13.75,25.750000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df['ide'] = (df['Politics2_1'] + df['Politics2_9'])/2

# Per-country median split
new = []
for _, sub in df.groupby("Country"):
    sub.loc[:, "Ideology"] = (sub.loc[:, "ide"] > sub.loc[:, "ide"].median()).astype(float)
    # Ensure NaN ideology when Politics2_1 is NaN
    sub.loc[sub.Politics2_1.isna(), "Ideology"] = np.NaN
    new.append(sub)
df = pd.concat(new)
# Replace 1 with "Conservative" and 0 with "Liberal"
df['Ideology'] = df['Ideology'].replace(1, 'Conservative').replace(0, 'Liberal')

df_control = df.query('cond==1')

In [4]:
import warnings
warnings.filterwarnings('ignore')

# Load R in Python
%load_ext rpy2.ipython

In [5]:
# Load the R library lmerTest
%R library(lmerTest)
%R library(ordinal)

R[write to console]: Loading required package: lme4

R[write to console]: Loading required package: Matrix

R[write to console]: 
Attaching package: ‘lmerTest’


R[write to console]: The following object is masked from ‘package:lme4’:

    lmer


R[write to console]: The following object is masked from ‘package:stats’:

    step




array(['ordinal', 'lmerTest', 'lme4', 'Matrix', 'tools', 'stats',
       'graphics', 'grDevices', 'utils', 'datasets', 'methods', 'base'],
      dtype='<U9')

# BASELINE BELIEFS

In [7]:
dfc_belief = pd.melt(
    df_control.loc[:, ['ResponseId', 'Country', 'condName', 'ide','Ideology','Income','Edu','Age','Gender', "WEPTcc",'Belief1', 'Belief2', 'Belief3', 'Belief4']],
    id_vars=['ResponseId', 'Country', 'condName', 'ide','Ideology','Income','Edu','Age','Gender', 'WEPTcc'],
    var_name="Item",
    value_name="Belief"
)
dfc_belief

,ResponseId,Country,condName,ide,Ideology,Income,Edu,Age,Gender,WEPTcc,Item,Belief
0,R_10Dy5kIdZiCS1Hh,Algeria,Control,NaN,NaN,NaN,2.0,23.0,1.0,2,Belief1,12.0
1,R_12eZDYyqMuHMs40,Algeria,Control,54.0,Liberal,1.0,3.0,20.0,2.0,7,Belief1,58.0
2,R_1DLU6t5YiTJMs5O,Algeria,Control,80.5,Conservative,2.0,3.0,34.0,1.0,8,Belief1,81.0
3,R_1FySakGF115qvHJ,Algeria,Control,54.0,Liberal,3.0,2.0,41.0,1.0,8,Belief1,72.0
4,R_1GP0L143l7yV8M1,Algeria,Control,56.0,Liberal,3.0,2.0,25.0,1.0,7,Belief1,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...
18487,R_pz6jT5cZRhgCzXX,Vietnam,Control,64.5,Conservative,1.0,3.0,20.0,1.0,8,Belief4,100.0
18488,R_Rqom9DZcYCveM49,Vietnam,Control,51.0,Liberal,1.0,3.0,19.0,1.0,3,Belief4,58.0
18489,R_SGM7rpBsClZHZ4Z,Vietnam,Control,NaN,NaN,9.0,3.0,21.0,1.0,8,Belief4,54.0
18490,R_Y5BtwTC9yOO8Rxf,Vietnam,Control,50.0,Liberal,1.0,2.0,19.0,2.0,8,Belief4,93.0


In [8]:
dfc_belief = dfc_belief.dropna()

In [19]:
# Send R the dataset
%Rpush dfc_belief

In [10]:
%%R

M1 <- lmer(Belief ~ Ideology + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_belief)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ Ideology + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: dfc_belief

REML criterion at convergence: 132132.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.2464 -0.2887  0.0866  0.3204  5.9098 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 519.2364 22.7868 
 Country    (Intercept)  33.1936  5.7614 
 Item       (Intercept)   0.3565  0.5971 
 Residual               148.0713 12.1685 
Number of obs: 15511, groups:  ResponseId, 3881; Country, 57; Item, 4

Fixed effects:
                 Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)       76.1549     1.0268   72.0089  74.168   <2e-16 ***
IdeologyLiberal    7.2150     0.7605 3843.8292   9.488   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
IdeolgyLbrl -0.382


# POLICY

In [11]:
dfc_policy = pd.melt(
    df_control.loc[:, ['ResponseId', 'Country', 'condName', 'ide','Ideology', "WEPTcc",'Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9']],
    id_vars=['ResponseId', 'Country', 'condName', 'ide','Ideology', 'WEPTcc'],
    var_name="Item",
    value_name="Policy"
)
dfc_policy

,ResponseId,Country,condName,ide,Ideology,WEPTcc,Item,Policy
0,R_10Dy5kIdZiCS1Hh,Algeria,Control,NaN,NaN,2,Policy1,19.0
1,R_12eZDYyqMuHMs40,Algeria,Control,54.0,Liberal,7,Policy1,100.0
2,R_1DLU6t5YiTJMs5O,Algeria,Control,80.5,Conservative,8,Policy1,90.0
3,R_1FySakGF115qvHJ,Algeria,Control,54.0,Liberal,8,Policy1,2.0
4,R_1GP0L143l7yV8M1,Algeria,Control,56.0,Liberal,7,Policy1,50.0
...,...,...,...,...,...,...,...,...
41602,R_pz6jT5cZRhgCzXX,Vietnam,Control,64.5,Conservative,8,Policy9,80.0
41603,R_Rqom9DZcYCveM49,Vietnam,Control,51.0,Liberal,3,Policy9,NaN
41604,R_SGM7rpBsClZHZ4Z,Vietnam,Control,NaN,NaN,8,Policy9,9.0
41605,R_Y5BtwTC9yOO8Rxf,Vietnam,Control,50.0,Liberal,8,Policy9,22.0


In [12]:
dfc_policy = dfc_policy.dropna()

In [13]:
# Send R the dataset
%Rpush dfc_policy

In [14]:
%%R

M1 <- lmer(Policy ~ Ideology + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_policy)
print(summary(M1))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ Ideology + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: dfc_policy

REML criterion at convergence: 310190.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5620 -0.5322  0.0622  0.6259  4.2566 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 315.49   17.762  
 Country    (Intercept)  19.34    4.398  
 Item       (Intercept) 157.91   12.566  
 Residual               411.99   20.297  
Number of obs: 34106, groups:  ResponseId, 3880; Country, 57; Item, 9

Fixed effects:
                 Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)       68.2061     4.2581    8.5256  16.018 1.17e-07 ***
IdeologyLiberal    4.0050     0.6142 3836.0422   6.521 7.91e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
IdeolgyLbrl -0.074


# BEHAVIOR

In [6]:
%Rpush df_control

In [7]:
%%R

M <- clmm(as.factor(WEPTcc) ~ Ideology + (1|Country), data = df_control, threshold = "equidistant",  doFit = TRUE)
summary(M)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ Ideology + (1 | Country)
data:    df_control

 link  threshold   nobs logLik   AIC      niter    max.grad cond.H 
 logit equidistant 3979 -6275.89 12559.79 164(498) 3.78e-03 8.5e+02

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.1734   0.4165  
Number of groups:  Country 57 

Coefficients:
                Estimate Std. Error z value Pr(>|z|)
IdeologyLiberal -0.04279    0.06103  -0.701    0.483

Threshold coefficients:
             Estimate Std. Error z value
threshold.1 -1.424168   0.077662  -18.34
spacing      0.185285   0.004893   37.87
(644 observations deleted due to missingness)
